In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


In [6]:
df_movies= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\A-ETLconsignas\MoviesSMALL.csv')
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44936 entries, 0 to 44935
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             44936 non-null  float64
 1   id                 44936 non-null  int64  
 2   original_language  44926 non-null  object 
 3   overview           44029 non-null  object 
 4   popularity         44936 non-null  float64
 5   release_date       44936 non-null  object 
 6   revenue            44936 non-null  float64
 7   runtime            44696 non-null  float64
 8   title              44936 non-null  object 
 9   release_year       44936 non-null  float64
 10  return             10879 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 3.8+ MB


In [7]:
df_movies["overview"] = df_movies["overview"].astype(str)

In [8]:
df_movies['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [9]:
columnas_elim=['budget','release_year', 'release_date', 'runtime', 'revenue', 'return','original_language']
df_movies2 = df_movies.drop(columns=columnas_elim)
df_movies_sorted= df_movies2.sort_values(ascending=False, by='popularity')

In [10]:
df_top_5= df_movies_sorted.head(15000) 

In [11]:
df_top_5.head()

,id,overview,popularity,title
30443,211672,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,Minions
33060,297762,An Amazon princess comes to the world of Man t...,294.337037,Wonder Woman
41779,321612,A live-action adaptation of Disney's version o...,287.253654,Beauty and the Beast
43170,339403,After being coerced into working for a crime b...,228.032744,Baby Driver
24267,177572,The special bond that develops between plus-si...,213.849907,Big Hero 6


In [12]:
df_top_5.to_csv('Top15kpelis.csv', index=False)

In [13]:
## Defino un objeto vectorizador TF-IDF. Elimino las palabras vacias en infles como 'the', 'a'
tfidf= TfidfVectorizer(stop_words='english')


In [14]:
df_movies2['overview'] = df_top_5['overview'].fillna('')

tfidf_matrix = tfidf.fit_transform(df_top_5['overview'])

tfidf_matrix.shape

(15000, 38004)

usaremos linear_kernel()

In [15]:
#Importamos
from sklearn.metrics.pairwise import linear_kernel

#Hacemos la matriz de la similaridad del coseno
cosine_sim= linear_kernel(tfidf_matrix, tfidf_matrix)

In [16]:
## Mecanismo para identificar el indice de una pelicula en nuestro DF dado su titulo
indices=pd.Series(df_top_5.index, index=df_top_5['title']).drop_duplicates()

In [17]:
## Definimos una funcion que tome el titulo de una pelicula como entrada y genere una lista de las 5 peliculas mas parecidas
def recomendaciones (title, cosine_sim=cosine_sim):
    idx= indices[title]
    
    sim_scores= list(enumerate(cosine_sim[idx]))
    
    sim_scores= sorted(sim_scores, key=lambda x : x[1], reverse=True)
    
    sim_scores=sim_scores[1:6]
    
    movies_indices= [i[0] for i in sim_scores]
    
    return df_top_5['title'].iloc[movies_indices]


In [18]:
recomendaciones('Toy Story')

21011                          Despicable Me 2
8002     Sherlock Holmes and the Secret Weapon
44615                                   Banana
5319                           Stuart Little 2
25987        Stuart Little 3: Call of the Wild
Name: title, dtype: object

In [278]:
### incorporo mas metricas 
## genre, cast, director, description, title

In [19]:
df_top_5.head()

,id,overview,popularity,title
30443,211672,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,Minions
33060,297762,An Amazon princess comes to the world of Man t...,294.337037,Wonder Woman
41779,321612,A live-action adaptation of Disney's version o...,287.253654,Beauty and the Beast
43170,339403,After being coerced into working for a crime b...,228.032744,Baby Driver
24267,177572,The special bond that develops between plus-si...,213.849907,Big Hero 6


In [20]:
df_top_5.shape

(15000, 4)

In [21]:
df_genero=pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\DESANIDANDO\GenerosAgrupados.csv') 
df_genero.head()

,id,name_genres
0,2,"Drama,Crime"
1,3,"Drama,Comedy"
2,5,"Crime,Comedy"
3,6,"Action,Thriller,Crime"
4,11,"Adventure,Action,Science Fiction"


In [22]:
df_cast= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\DESANIDANDO\ActoresPorPEliID.csv')
df_cast.rename(columns={"id_original": "id"}, inplace=True)
df_cast.drop(columns='name', inplace=True)
df_cast.head()

,id,character
0,2,"Turo Pajala, Susanna Haavisto, Matti Pellonpä..."
1,3,"Matti Pellonpää, Kati Outinen, Sakari Kuosman..."
2,5,"Tim Roth, Antonio Banderas, Jennifer Beals, M..."
3,6,"Emilio Estevez, Cuba Gooding Jr., Denis Leary..."
4,11,"Mark Hamill, Harrison Ford, Carrie Fisher, Pe..."


In [23]:
df_directores= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\DESANIDANDO\directoresAgrupadosID.csv')
df_directores.rename(columns={"id_original": "id"}, inplace=True)
df_directores.head()

,id,name
0,100,Guy Ritchie
1,10000,Sergio Cabrera
2,10001,Yahoo Serious
3,100010,Frank Borzage
4,100017,Angelina Maccarone


In [24]:
df_concat= pd.merge(df_top_5,df_cast, on='id', how='left')
df_concat2= pd.merge(df_concat, df_directores, on='id', how ='left')
df_concat3= pd.merge(df_concat2, df_genero, on='id', how='left')


In [25]:
df_concat3.head()


,id,overview,popularity,title,character,name,name_genres
0,211672,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,Minions,"Sandra Bullock, Jon Hamm, Michael Keaton, All...","Kyle Balda,Pierre Coffin","Family,Animation,Adventure,Comedy"
1,297762,An Amazon princess comes to the world of Man t...,294.337037,Wonder Woman,"Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",Patty Jenkins,"Action,Adventure,Fantasy"
2,321612,A live-action adaptation of Disney's version o...,287.253654,Beauty and the Beast,"Emma Watson, Dan Stevens, Luke Evans, Kevin K...",Bill Condon,"Family,Fantasy,Romance"
3,339403,After being coerced into working for a crime b...,228.032744,Baby Driver,"Ansel Elgort, Lily James, Kevin Spacey, Jamie...",Edgar Wright,"Action,Crime"
4,177572,The special bond that develops between plus-si...,213.849907,Big Hero 6,"Scott Adsit, Ryan Potter, Daniel Henney, T.J....","Chris Williams,Don Hall","Adventure,Family,Animation,Action,Comedy"


In [26]:
df_concat3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           15000 non-null  int64  
 1   overview     15000 non-null  object 
 2   popularity   15000 non-null  float64
 3   title        15000 non-null  object 
 4   character    15000 non-null  object 
 5   name         14978 non-null  object 
 6   name_genres  14982 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 820.4+ KB


In [28]:
df_concat3.to_csv('FuncionML.csv', index=False)

In [29]:
###Ahora con el dataframe con todas las caracteristicas que necesito armo otra funcion

elim= ['id','popularity']
filters= df_concat3.drop(columns=elim)
filters.head()

,overview,title,character,name,name_genres
0,"Minions Stuart, Kevin and Bob are recruited by...",Minions,"Sandra Bullock, Jon Hamm, Michael Keaton, All...","Kyle Balda,Pierre Coffin","Family,Animation,Adventure,Comedy"
1,An Amazon princess comes to the world of Man t...,Wonder Woman,"Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",Patty Jenkins,"Action,Adventure,Fantasy"
2,A live-action adaptation of Disney's version o...,Beauty and the Beast,"Emma Watson, Dan Stevens, Luke Evans, Kevin K...",Bill Condon,"Family,Fantasy,Romance"
3,After being coerced into working for a crime b...,Baby Driver,"Ansel Elgort, Lily James, Kevin Spacey, Jamie...",Edgar Wright,"Action,Crime"
4,The special bond that develops between plus-si...,Big Hero 6,"Scott Adsit, Ryan Potter, Daniel Henney, T.J....","Chris Williams,Don Hall","Adventure,Family,Animation,Action,Comedy"


In [30]:
filters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   overview     15000 non-null  object
 1   title        15000 non-null  object
 2   character    15000 non-null  object
 3   name         14978 non-null  object
 4   name_genres  14982 non-null  object
dtypes: object(5)
memory usage: 586.1+ KB


In [31]:
# Me aseguro que todas las columnas sean str para que no me de error la funcion
for column in filters.columns:
    filters[column] = filters[column].astype(str)

In [32]:
#elimino espacios entre texto y mayusculas
def clean_data(x):
    return str.lower(x.replace(" ",""))


In [33]:
#aplico la funcion
for column in filters.columns:
    filters[column] = filters[column].apply(clean_data)


In [34]:
filters.head()

,overview,title,character,name,name_genres
0,"minionsstuart,kevinandbobarerecruitedbyscarlet...",minions,"sandrabullock,jonhamm,michaelkeaton,allisonjan...","kylebalda,pierrecoffin","family,animation,adventure,comedy"
1,anamazonprincesscomestotheworldofmantobecometh...,wonderwoman,"galgadot,chrispine,robinwright,dannyhuston,dav...",pattyjenkins,"action,adventure,fantasy"
2,alive-actionadaptationofdisney'sversionofthecl...,beautyandthebeast,"emmawatson,danstevens,lukeevans,kevinkline,jos...",billcondon,"family,fantasy,romance"
3,"afterbeingcoercedintoworkingforacrimeboss,ayou...",babydriver,"anselelgort,lilyjames,kevinspacey,jamiefoxx,jo...",edgarwright,"action,crime"
4,thespecialbondthatdevelopsbetweenplus-sizedinf...,bighero6,"scottadsit,ryanpotter,danielhenney,t.j.miller,...","chriswilliams,donhall","adventure,family,animation,action,comedy"


Ahora creamos un conjunto de metadatos

In [35]:
def crear_datos(x):
    return x['name'] + ' ' + x['character']+ ' ' + x['name_genres']+ ' ' + x['overview']


In [36]:
filters['metadatos']= filters.apply(crear_datos, axis=1)

In [45]:
filters.to_csv('filters.csv', index=False)

In [37]:
## Mismo pasos que la recomendacion anterior pero usamos Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

count= CountVectorizer(stop_words='english')
count_matrix= count.fit_transform(filters['metadatos'])

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2= cosine_similarity(count_matrix, count_matrix)

In [39]:
##Reestrablezco el indice de Df inicial y hago le mapeo inverso
filters= filters.reset_index()
indices=pd.Series(filters.index, index=filters['title'])

In [40]:
def recomendacion_peli(title, cosine_sim=cosine_sim2):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    title=title.replace(' ','').lower()
    idx= indices[title]
    
    sim_scores= list(enumerate(cosine_sim[idx]))
    
    sim_scores= sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    sim_scores= sim_scores[1:6]
    
    movie_indices= [i[0] for i in sim_scores]
    
    lista= df_concat3['title'].iloc[movie_indices]
    lista2= lista.to_list()
    return lista2
    

In [41]:
recomendacion_peli('toy story')


['Toy Story 2',
 'Toy Story 3',
 'Small Fry',
 'Toy Story That Time Forgot',
 'Hawaiian Vacation']

In [42]:
recomendacion_peli('Toy Story')

['Toy Story 2',
 'Toy Story 3',
 'Small Fry',
 'Toy Story That Time Forgot',
 'Hawaiian Vacation']

In [43]:
recomendacion_peli('Minions')

['Banana',
 'Minions: Orientation Day',
 'Despicable Me 3',
 'Mower Minions',
 'Binky Nelson Unpacified']